# Dogs vs Cats

[Kaggle](https://www.kaggle.com/c/dogs-vs-cats)

1 = dog

0 = cat

In [30]:
import cv2
from matplotlib import pyplot as plt
import sklearn
import numpy as np

plt.style.use('ggplot')
%matplotlib inline

### Load training dataset

In [18]:
train_folder = 'data/train'

In [24]:
def load_image(filepath, gray=False):
    img = cv2.imread(filepath)
    
    if gray:
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        return img

In [28]:
from os import listdir

imgs_paths = [train_folder + '/' + filepath for filepath in listdir(train_folder)]

In [29]:
imgs = [load_image(img_path, gray=True) for img_path in imgs_paths]

In [33]:
imgs = np.array(imgs)

In [34]:
print(imgs.shape)

(25000,)


In [35]:
labels = [1 if "dog" in path else 0 for path in imgs_paths]

### Features Extraction